In [ ]:
!pwd

In [1]:
!python --version

Python 3.7.6


# 修改checkpoint变量名

In [3]:
import os
from tensorflow.python import pywrap_tensorflow
model_dir="/root/magenta/magenta/models/rl_tuner/checkpoint/src/model.ckpt-15321" #checkpoint的文件位置
# Read data from checkpoint file
reader = pywrap_tensorflow.NewCheckpointReader(model_dir)
var_to_shape_map = reader.get_variable_to_shape_map()
# Print tensor name and values
for key in var_to_shape_map:
    print("tensor_name: ", key)  #输出变量名
    #print(reader.get_tensor(key))   #输出变量值

tensor_name:  rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel/Adam_1
tensor_name:  rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel
tensor_name:  rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias/Adam_1
tensor_name:  rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/kernel/Adam_1
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/kernel
tensor_name:  rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias/Adam
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/attention/attn_w/Adam_1
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/attention/attn_w
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/attention/attn_v
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/attention/attn_w/Adam
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/attention/bias/Adam_1
tensor_name:  fully_connected/biases
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/basic_ls

In [7]:
!python /root/magenta/magenta/models/rl_tuner/rename_tf_variable.py \
--ckpt_path /root/magenta/magenta/models/rl_tuner/checkpoint/src \
--save_path /root/magenta/magenta/models/rl_tuner/checkpoint/model.ckpt-15321 \
--rename_var_src fully_connected/weights,fully_connected/biases \
--rename_var_dst rnn_model/fully_connected/weights,rnn_model/fully_connected/bias


2020-04-02 12:47:50.693803: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2020-04-02 12:47:50.700892: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2500015000 Hz
2020-04-02 12:47:50.701129: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559038848510 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-04-02 12:47:50.701154: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Renaming beta1_p

In [9]:
import os
from tensorflow.python import pywrap_tensorflow
model_dir="/root/magenta/magenta/models/rl_tuner/checkpoint/model.ckpt-15321" #checkpoint的文件位置
# Read data from checkpoint file
reader = pywrap_tensorflow.NewCheckpointReader(model_dir)
var_to_shape_map = reader.get_variable_to_shape_map()
# Print tensor name and values
for key in var_to_shape_map:
    print("tensor_name: ", key)  #输出变量名
    #print(reader.get_tensor(key))   #输出变量值

tensor_name:  rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel/Adam_1
tensor_name:  rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel
tensor_name:  rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias/Adam_1
tensor_name:  rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/kernel/Adam_1
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/kernel
tensor_name:  rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias/Adam
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/attention/attn_w/Adam_1
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/attention/attn_w
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/attention/attn_v
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/attention/attn_w/Adam
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/attention/bias/Adam_1
tensor_name:  fully_connected/biases
tensor_name:  rnn/multi_rnn_cell/cell_0/attention_cell_wrapper/basic_ls

In [ ]:
import tensorflow as tf
import numpy as np
import sys
import importlib

In [ ]:
from magenta.models.rl_tuner import rl_tuner
from magenta.models.rl_tuner import rl_tuner_ops

In [ ]:
#设定输出文件存储路径
SAVE_PATH = "/root/magenta/magenta/models/rl_tuner/output/2020.4.1"

In [ ]:
# Model parameter settings
ALGORITHM = 'q'
REWARD_SCALER = 1
OUTPUT_EVERY_NTH = 50000
NUM_NOTES_IN_COMPOSITION = 64
PRIME_WITH_MIDI = False
NOTE_RNN_CHECKPOINT_DIR = '/root/magenta/magenta/models/rl_tuner/checkpoint'
NOTE_RNN_TYPE = 'default'

In [ ]:
rl_tuner_hparams = tf.contrib.training.HParams(random_action_probability=0.1,
                                               store_every_nth=1,
                                               train_every_nth=5,
                                               minibatch_size=32,
                                               discount_rate=0.5,
                                               max_experience=100000,
                                               target_network_update_rate=0.01)

# Train network

In [ ]:
importlib.reload(sys)
importlib.reload(rl_tuner_ops)
importlib.reload(rl_tuner)
rl_tuner.reload_files()

In [ ]:
rl_net = rl_tuner.RLTuner(SAVE_PATH, 
                          dqn_hparams=rl_tuner_hparams, 
                          algorithm=ALGORITHM,
                          reward_scaler=REWARD_SCALER,
                          output_every_nth=OUTPUT_EVERY_NTH,
                          num_notes_in_melody=NUM_NOTES_IN_COMPOSITION,
                          note_rnn_checkpoint_dir=NOTE_RNN_CHECKPOINT_DIR)

In [ ]:
# Generate initial music sequence before training with RL
rl_net.generate_music_sequence(visualize_probs=True, title='pre_rl', length=32)

In [ ]:
rl_net.train(num_steps=1000000, exploration_period=500000)

In [ ]:
# Plot the rewards received during training. Improves as chance of random exploration action decreases.
rl_net.plot_rewards()

In [ ]:
# Plot rewards received during calls to evaluation function throughout training. 
# Does not include exploration or random actions.
rl_net.plot_evaluation()